### 2.3.1 概念

`Tensor`类是`torch`包的核心类，如果将其属性`.requires_grad`设置为`True`，它将开始追踪(trace)在其上的所有操作(这样就可以利用链式法则进行梯度传播了)。完成计算后，可以调用`.backward()`来完成所有梯度计算。此`Tensor`的梯度将累积到`.grad`属性中。

> 注意在`y.backward()`时，如果`y`是标量，则不需要为`backward()`传入任何参数；否则，需要传入一个与`y`同形的`Tensor`。

如果不想要呗继续追踪，可以调用`.detach()`将其从追踪记录中分离出来，这样就可以放置将来的计算被追踪，这样梯度就传不过去了。可以用`with torch.no_grad():`将不想被追踪的操作代码包裹起来，这种方法在评估模型时很常用，因为在评估模型时，不需要计算可训练参数(`requires_grad=True`)的梯度。

`Function`是另一个很重要的类。`Tensor`和`Function`相互结合就可以构建一个记录有整个计算过程的有向无环图(DAG)。每个`Tensor`都有一个`.grad_fn`属性，该属性即创建该`Tensor`的`Function`，就是说该`Tensor`是不是通过某些运算得到的(还是用户直接创建的)，若是，则`grad_fn`返回一个与这些运算相关的对象，否则是`None`。

In [10]:
import torch

x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


因为x是直接创建的，所以它没有`grad_fn`，而y是由一个加法操作创建的，所以它有一个为`<AddBackward>`的`grad_fn`。

像x这种直接创建的称为**叶子节点**，叶子节点对应的`grad_fn`是`None`。

In [11]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [12]:
print(x.is_leaf, y.is_leaf)

True False


In [13]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


通过`.requires_grad_()`来用`in-place`的方式改变`requires_grad`属性

In [14]:
a = torch.randn(2, 2)   # 默认情况下 requires_grad=False
a = (a * 3) / (a - 1)
print(a.requires_grad)  # False
a.requires_grad_(True)
print(a.requires_grad)  # True
b = (a * a).sum()
print(b.grad_fn)

False
True


### 2.3.3 梯度

因为`out`是一个标量，所以调用`backward()`时不需要指定求导变量：

In [15]:
out.backward()  # 等价于 out.backward(torch.tensor(1.))

In [16]:
# `out`关于`x`的梯度
# out = 3/4 * (x + 2) ^ 2
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


> 注意：`grad`在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需要把梯度清零。

In [17]:
# 再来反向传播一次，注意grad是累加的
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


我们不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量。所以必要时我们把张量通过将所有张量元素加权求和方式转换为标量。

例如，假设`y`由自变量`x`计算而来，`w`是和`y`同形的张量，则`y.backward(w)`的含义是：先计算`l = torch.sum(y * w)`，则`l`是个标量，然后求`l`对自变量`x`的导数。

In [18]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)


现在`z`不是一个标量，所以在调用`backward`时需要传入一个和`z`同形的权重向量进行加权求和得到一个标量。

In [20]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


> 注意：`x.grad`是和`x`同形的张量。

如下是中断梯度追踪的例子：

In [21]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


可以看到，上面的`y2`是没有`grad_fn`而且`y2.requires_grad=False`的，而`y3`是有`grad_fn`的。

In [22]:
y3.backward()
print(x.grad)

tensor(2.)


由于`y2`的定义是在`torch.no_grad():`包裹中，所以与`y2`有关的梯度不会回传，只有与`y1`有关的梯度才会回传，即`x^2`对`x`的梯度。

上面提到，`y2.requires_grad=False`，所以不能调用`y2.backward()`，会报错。

此外，如果想修改`tensor`的值，但是又不希望被`autograd`记录(即不会影响反向传播)，可以对`tensor.data`进行操作。

In [23]:
x = torch.ones(1, requires_grad=True)
print(x.data)   # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外

y = 2 * x
x.data *= 100   # 只改变值，不会记录在计算图，所以不会影响梯度传播

y.backward()
print(x)        # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
